In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="TopicClusterMapping.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_58797/4032682437.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [2]:
import sys

sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/')

from src.clustering.ClustersResetService import ClustersResetService
from src.clustering.ClusterAssignmentService import ClusterAssignmentService
from sql.clustering.ClusteringSQL import ClusteringSQL
from src.onboarding.TopicClusterMapping import TopicClusterMapping

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [3]:
from sql.embeddings.EmbeddingSQL import EmbeddingSQL

In [4]:
all_articles = EmbeddingSQL.get_article_ids_with_embeddings()

In [6]:
num_users = 20

In [18]:
num_days = 60

In [43]:
feed_ids = list(range(1000,4000))

In [8]:
import numpy as np

In [14]:
np.random.rand()

0.21304821720445577

In [19]:
from datetime import timedelta

In [20]:
from datetime import datetime

In [25]:
datetime.today() - timedelta(days = 3)

datetime.datetime(2023, 9, 26, 14, 39, 34, 917780)

In [44]:
feed_content = []
for feed_id in feed_ids:
    feed_date = datetime.now() - timedelta(days=np.random.randint(1,60), hours = np.random.randint(1,23), minutes = np.random.randint(1,60))
    num_articles = np.random.randint(3, 20)
    user_id = np.random.randint(1,50)
    articles = np.random.choice(all_articles, size = num_articles, replace = False)
    click_ratio = np.random.rand()
    is_opened = [True if np.random.rand() < click_ratio else False for _ in range(num_articles)]
    for i in range(num_articles):
        feed_content.append((user_id, feed_id, articles[i], is_opened[i], i, feed_date))

In [72]:
from sql.DatabaseOperation import DatabaseOperation

In [48]:
def insert_into_db(event):
    with DatabaseOperation() as cursor:
        sql = f"""INSERT INTO user_article_interactions (user_id, feed_id, article_id, is_article_opened, article_position, created_at)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (event[0], event[1], event[2], event[3], event[4], event[5]))

In [49]:
from tqdm.notebook import tqdm

In [56]:
feeds = feed_content[103:]

In [59]:
len(feeds)

33531

In [60]:
for eve in tqdm(feeds[20:]):
    insert_into_db(event=eve)

  0%|          | 0/33511 [00:00<?, ?it/s]

In [28]:
len(feed_content)

33293

### Candidate articles

In [65]:
len(all_articles)

8390

In [68]:
candidate_articles = np.random.choice(all_articles,7000,replace = False)

In [69]:
time = datetime.now()

In [75]:
def insert_candidate_article(aid):
    with DatabaseOperation() as cursor:
        sql = f"""INSERT INTO candidate_articles (article_id, created_at)
            VALUES (%s, %s)
            """
        cursor.execute(sql, (aid, time))

In [77]:
for art in tqdm(candidate_articles):
    insert_candidate_article(aid=art)

  0%|          | 0/7000 [00:00<?, ?it/s]